### Import required libraries

In [1]:
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import subprocess as sp
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical


2022-12-15 14:38:52.294647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/link-lap-24/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-15 14:38:52.294674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=9120)])
#     except RuntimeError as e:
#         print(e)

### Read videos

In [2]:
IMG_SIZE = 224
MAX_SEQ_LENGTH = 16
WINDOW_SIZE = 2

In [112]:
def load_video(path, event_time, window_size=WINDOW_SIZE, max_frames=MAX_SEQ_LENGTH, resize=(IMG_SIZE, IMG_SIZE)):
    # get frame dimensions from video
    path1 = path.split('/')[-1].split('.')[0]
    command = [ 'ffprobe','-v', 'error','-select_streams', 'v:0','-show_entries', 'stream=width,height,duration',
                '-of', 'json', path ]
    pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=20**5)
    video_info = json.loads(pipe.stdout.read())
    print("video_info:",video_info)
    width = video_info['streams'][0]['width']
    height = video_info['streams'][0]['height']
    try:
        duration = video_info['streams'][0]['duration']
        print("duration of video::",duration)
    except Exception as e:
        print(e)
    pipe.stdout.flush()
    
    start_times = []
    frame_seq = []
    fps = int(MAX_SEQ_LENGTH/window_size)
    
    event_time =51.7
    print("event time::",event_time)
    if event_time < window_size:
        print("window size greater than event time")
        return start_times
    start_times.append(event_time - window_size)
    print("start_times before event:;",start_times)
    for start_time in start_times:
        #reading video event at given time
        command = [ 'ffmpeg','-loglevel', 'quiet','-ss', str(start_time),'-t', str(window_size),'-i', path,
                    # '-r', str(fps),
                    '-vf', 'fps={}'.format(fps),
                    '-f', 'image2pipe',
                    '-pix_fmt', 'rgb24',
                    '-vcodec', 'rawvideo', '-' ]
        pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=10**5)

        frames = []
        try:
            while True:
                raw_image = pipe.stdout.read(width*height*3)
                if not raw_image:
                    break
                frame =  np.frombuffer(raw_image, dtype='uint8')
                frame = frame.reshape((height,width,3))
                frame = cv2.resize(frame, resize)
                #print("type of frame::",type(frame))
                frames.append(frame)
                if len(frames) == max_frames:
                    break
        finally:
            pipe.stdout.flush()
        frame_seq.append(np.array(frames))
        
        print("len of frame_seq::",len(np.array(frames)))
        for idx, frame in enumerate(frames):
            #image = cv2.imread(frame)
            #print("frame shape::",frame.shape)
#             if(len(frame.shape)<3):
#                   print('gray')
#             elif len(frame.shape)==3:
#                   print ('Color(RGB)')
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Use the cvtColor() function to grayscale the image
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #cv2.imwrite(os.path.join('seq_frames','input_frames_train', f'{path1}_{idx+1}.png'), frame)
            cv2.imwrite(os.path.join('seq_frames','input_frames_train', f'{idx+1}.png'), frame)
    

    return frame_seq
def prepare_all_videos(df):
    input_data, labels = [], []
    with tqdm(total=len(df), desc="Progress") as pbar:
        for idx, video_name, label, event_time in df.itertuples():
            #video_path = os.path.join(root_dir,video_name)
            #video_path=video_name
            video_path = "/home/link-lap-24/Downloads/tagging/ICC/clips/262.mp4"
            print("video_path::",video_path)
            frame_seq = load_video(video_path, event_time)
            if frame_seq:
                for frame_arr in frame_seq:
                    #print("frame_arr shape::",frame_arr.shape) nd array
                    input_data.append(frame_arr)                    
                    labels.append(label)
                    
            pbar.update(1)
            break
    #input_data = np.array(input_data,dtype=object)
    input_data=np.array(input_data).astype('float32')


    labels = np.array(labels)
    return input_data, labels

# load video data
df = pd.read_csv('/home/link-lap-24/Downloads/tagging/ICC/Updated_ICC.csv')
print("length of df::",len(df))
#data_dir = os.path.join('test/data1', 'videos')
input_data, labels = prepare_all_videos(df)

print(f"Data Shape: {input_data.shape}")
print(f"Labels Shape: {labels.shape}")

length of df:: 189


Progress:   0%|                                         | 0/189 [00:00<?, ?it/s]

video_path:: /home/link-lap-24/Downloads/tagging/ICC/clips/262.mp4
video_info: {'programs': [], 'streams': [{'width': 1280, 'height': 720, 'duration': '196.240000'}]}
duration of video:: 196.240000
event time:: 51.7
start_times before event:; [49.7]


Progress:   1%|▏                                | 1/189 [00:00<01:01,  3.06it/s]

len of frame_seq:: 16
Data Shape: (1, 16, 224, 224, 3)
Labels Shape: (1,)


### Save/Load train data

In [ ]:
#save train data
np.savez_compressed('array_data/train_data_celeb_no_celeb_dec_14.npz', input_data, labels)



In [4]:
#load  data
#npz contains collection of numpy arrays
data = np.load('array_data/train_data_celeb_no_celeb_dec_14.npz')
#p = np.random.permutation(len(data['arr_1']))
# for i in data:
#     print(i)
train_data = data['arr_0']
train_labels = data['arr_1']


In [5]:
p = np.random.permutation(len(train_labels))
train_data = train_data[p]
train_labels = train_labels[p]

### Data preprocessing

In [6]:
train_data = train_data/255
#train_labels = train_labels[...,None]
train_labels = to_categorical(train_labels,num_classes=2)
print(f"train data Shape: {train_data.shape}")
print(f"Labels Shape: {train_labels.shape}")
CLASSES_LIST = ["Batsman_Celebration","No_Celebration"]

train data Shape: (527, 16, 224, 224, 3)
Labels Shape: (527, 2)


### Define model

In [7]:
def get_lstm_model(seq_length, img_size):
    np.random.seed(1234)
    base_model = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))
    # Freeze all the layers
    for layer in base_model.layers:
        layer.trainable = False
    
    cnn = keras.models.Sequential()
    cnn.add(base_model)
    cnn.add(keras.layers.Flatten())
    
    # define LSTM model
    model = keras.models.Sequential()
    model.add(keras.layers.TimeDistributed(cnn,  input_shape=(seq_length, img_size, img_size, 3)))
    model.add(keras.layers.LSTM(40, return_sequences=True))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(160, activation='relu')))
    model.add(keras.layers.GlobalAveragePooling1D(name="global"))
    # model.add(keras.layers.Dropout(0.3))
    # model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(len(CLASSES_LIST), activation="sigmoid" , name="output"))

    adam = keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
    print(model.summary())
    return model

### Train model

In [8]:
BATCH_SIZE = 1
EPOCHS = 30
MODEL_PATH = 'weights/lstm_celeb_vs_noceleb_dec14_new_arch_vgg19.hdf5'

def fit_model():
    model = get_lstm_model(MAX_SEQ_LENGTH, IMG_SIZE)

    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=1e-5, verbose=0, mode='min')
    checkpoint = keras.callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=2, factor=0.5, min_lr=0.0000001)

    history = model.fit(train_data, train_labels, batch_size=BATCH_SIZE, validation_split=0.3,
                    epochs=EPOCHS, callbacks=[earlyStopping, checkpoint, reduce_lr_loss])
    return history, model

In [9]:
print(f"Data Shape: {train_data.shape}")
print(f"Labels Shape: {train_labels.shape}")
history, model = fit_model()

Data Shape: (527, 16, 224, 224, 3)
Labels Shape: (527, 2)


2022-12-14 20:20:35.441147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 20:20:35.462308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 20:20:35.462689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 20:20:35.464482: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 25088)         20024384  
_________________________________________________________________
lstm (LSTM)                  (None, 16, 40)            4020640   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 160)           6560      
_________________________________________________________________
global (GlobalAveragePooling (None, 160)               0         
_________________________________________________________________
dropout (Dropout)            (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              164864    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)             

2022-12-14 20:20:36.344356: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3545235456 exceeds 10% of free system memory.
2022-12-14 20:20:37.914447: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3545235456 exceeds 10% of free system memory.
2022-12-14 20:20:39.025603: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-12-14 20:20:40.591105: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


  3/368 [..............................] - ETA: 16s - loss: 0.7029 - accuracy: 0.3333      

Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda-11.1/lib64/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2022-12-14 20:20:42.464036: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


368/368 [==============================] - 29s 68ms/step - loss: 0.6938 - accuracy: 0.5000 - val_loss: 0.6930 - val_accuracy: 0.5283
Epoch 2/100
368/368 [==============================] - 24s 64ms/step - loss: 0.6943 - accuracy: 0.5163 - val_loss: 0.6930 - val_accuracy: 0.5849
Epoch 3/100
368/368 [==============================] - 24s 64ms/step - loss: 0.6948 - accuracy: 0.4565 - val_loss: 0.6933 - val_accuracy: 0.4717
Epoch 4/100
368/368 [==============================] - 24s 65ms/step - loss: 0.6938 - accuracy: 0.5082 - val_loss: 0.6933 - val_accuracy: 0.5283


### Training graphs

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.legend(["accuracy", "val_accuracy"], loc ="lower right")

plt.show()
#plt.savefig('accuracy_valaccurac.png')



In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.legend(["loss", "val_loss"], loc ="lower right")

plt.show()
#plt.savefig('loss_valloss.png')


In [10]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()